# KNN Simulation en utilisant l'information mutuelle

KNN Mutual Information - fonction distance_mi - puis simulation avec pondération du dataset


## Import des librairies

In [23]:
# !pip install geopandas
# !pip install --upgrade matplotlib
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline 
import seaborn as sns
from datetime import datetime, timezone, timedelta
from sklearn.metrics import confusion_matrix, classification_report
# import geopandas
from sklearn.linear_model import LinearRegression, LogisticRegression

from scipy.stats import chi2_contingency

# Import des librairies bokeh
from bokeh.models import ColumnDataSource, GeoJSONDataSource, HoverTool
from bokeh.plotting import figure, show
from bokeh.models.widgets import Panel, Tabs
from bokeh.layouts import row,column
from bokeh.io import push_notebook,output_notebook, show
output_notebook()  # pour un affichage dans le notebook

Loading BokehJS ...

##Chargement du fichier avec les variables issues de la phase preprocessing

In [24]:
# df_final = pd.read_csv(path, index_col=0)
df_final = pd.read_csv("C:\workspaces\Datascientest\MonProjetMeteo\data\weatherAUS_Kapy_v2_20230708.csv",
                       index_col=0,parse_dates=[1])

In [25]:
# Affichage du nombre de valeurs restantes avec un dropna
df_final.dropna(inplace=True)
df_final.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 119115 entries, 3 to 145458
Data columns (total 33 columns):
 #   Column                  Non-Null Count   Dtype         
---  ------                  --------------   -----         
 0   Date                    119115 non-null  datetime64[ns]
 1   Location                119115 non-null  object        
 2   Rainfall                119115 non-null  float64       
 3   WindGustSpeed           119115 non-null  float64       
 4   Temp9am                 119115 non-null  float64       
 5   RainTomorrow            119115 non-null  float64       
 6   DayOfYear               119115 non-null  int64         
 7   Month                   119115 non-null  int64         
 8   Latitude                119115 non-null  float64       
 9   Longitude               119115 non-null  float64       
 10  diffWind3pm9am          119115 non-null  float64       
 11  diffTempMinMax          119115 non-null  float64       
 12  diffPressure9am3pm      119115

# Suppression de quelques variables non utilisées

In [26]:
# suppression du lieu, et des coord géographiques
df = df_final.drop(['Date','Location','Latitude','Longitude'],axis=1)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 119115 entries, 3 to 145458
Data columns (total 29 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   Rainfall                119115 non-null  float64
 1   WindGustSpeed           119115 non-null  float64
 2   Temp9am                 119115 non-null  float64
 3   RainTomorrow            119115 non-null  float64
 4   DayOfYear               119115 non-null  int64  
 5   Month                   119115 non-null  int64  
 6   diffWind3pm9am          119115 non-null  float64
 7   diffTempMinMax          119115 non-null  float64
 8   diffPressure9am3pm      119115 non-null  float64
 9   Pressure                119115 non-null  float64
 10  DeltaP_1d               119115 non-null  float64
 11  DeltaP_2d               119115 non-null  float64
 12  DeltaP_3d               119115 non-null  float64
 13  diffHimidity9am3pm      119115 non-null  float64
 14  Humidity            

# Jeu de données d'entrainement et de test

In [27]:
# séparation en features et target
target = df["RainTomorrow"]
features = df.drop("RainTomorrow",axis=1)

features.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 119115 entries, 3 to 145458
Data columns (total 28 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   Rainfall                119115 non-null  float64
 1   WindGustSpeed           119115 non-null  float64
 2   Temp9am                 119115 non-null  float64
 3   DayOfYear               119115 non-null  int64  
 4   Month                   119115 non-null  int64  
 5   diffWind3pm9am          119115 non-null  float64
 6   diffTempMinMax          119115 non-null  float64
 7   diffPressure9am3pm      119115 non-null  float64
 8   Pressure                119115 non-null  float64
 9   DeltaP_1d               119115 non-null  float64
 10  DeltaP_2d               119115 non-null  float64
 11  DeltaP_3d               119115 non-null  float64
 12  diffHimidity9am3pm      119115 non-null  float64
 13  Humidity                119115 non-null  float64
 14  DeltaH_1d           

In [28]:
from sklearn.model_selection import train_test_split, KFold, GridSearchCV
from sklearn.linear_model import LinearRegression, Lasso
from sklearn.metrics import confusion_matrix

In [29]:
# séparation en jeux d'entrainement et de test

X_train, X_test, y_train, y_test = train_test_split(features, target, test_size = .2, random_state = 42, shuffle = True)
print(X_train.shape)
print(X_test.shape)
print(pd.DataFrame(y_test).value_counts())

(95292, 28)
(23823, 28)
RainTomorrow
0.0             18590
1.0              5233
dtype: int64


# **Normalisation des données**

Nous allons utiliser un preprocessing de normalisation MinMax.

La normalisation MinMax effectue le calcul suivant sur chaque variable :

X_std = (X - X.min(axis=0)) / (X.max(axis=0) - X.min(axis=0))

X_scaled = X_std * (max - min) + min

Les valeurs des variables sont donc ramenées entre 0 et 1.



In [30]:
from sklearn.preprocessing import MinMaxScaler

# pour plus d'info, voir : https://scikit-learn.org/stable/auto_examples/preprocessing/plot_all_scaling.html

minmax = MinMaxScaler()

X_train_minmax = minmax.fit_transform(X_train,y_train)

X_test_minmax = minmax.transform(X_test)

pd.DataFrame(X_train_minmax).describe().transpose().mean()

count    95292.000000
mean         0.321393
std          0.171141
min          0.000000
25%          0.226067
50%          0.279766
75%          0.409257
max          1.000000
dtype: float64

In [31]:
def Accuracy(TP,TN,FP,FN):
    return (TP+TN)/(TP+FP+FN+TN)

def Precision(TP,FP):
    return TP/(TP+FP)

def Recall(TP,FN):
    return TP/(TP+FN)

def F1score(Recall,Precision):
    return 2*(Recall * Precision) / (Recall + Precision)

def Specificity(TN,FP):
    return TN/(TN+FP)

##Première simulation

In [32]:
# Première simulation sur le jeu de données normalisées

from sklearn.neighbors import KNeighborsClassifier

# Information mutuelle

In [33]:
from sklearn.feature_selection import f_regression, mutual_info_regression

f_statistic, p_values = f_regression(X_train_minmax, y_train)
f_tests = f_statistic/max(f_statistic)

# Permet de déterminer sur une dizaine de tirages au sort
# la moyenne de l'information mutuelle
info_mutuelle = []
for i in range(10):
  info_mutuelle.append(mutual_info_regression(X_train_minmax, y_train))

In [36]:
info_mutuelle_moy = np.mean(info_mutuelle,axis=0)

mi_inv = 1/info_mutuelle_moy

data_info = {'Variable':features.columns,'F-test':f_tests,'Info Mutuelle':info_mutuelle_moy, 'mi_inv':mi_inv}
df_info = pd.DataFrame(data=data_info)
print(df_info.sort_values(['mi_inv'], ascending=[True]))
print("On peut voir que les variables sont classées de la même manière suivant que l'on utilise")
print("F-test ou L'Info Mutuelle")

                  Variable    F-test  Info Mutuelle       mi_inv
13                Humidity  1.000000       0.119809     8.346632
6           diffTempMinMax  0.506554       0.069007    14.491365
0                 Rainfall  0.228365       0.055154    18.131145
15               DeltaH_2d  0.359039       0.048385    20.667769
16               DeltaH_3d  0.367250       0.047508    21.049256
17        WindDirInfluence  0.438571       0.047071    21.244280
14               DeltaH_1d  0.305847       0.044229    22.609412
18  consecutiveRainingDays  0.275921       0.043429    23.026075
12      diffHimidity9am3pm  0.303117       0.038883    25.718215
8                 Pressure  0.211590       0.027490    36.376282
1            WindGustSpeed  0.229651       0.026535    37.685794
11               DeltaP_3d  0.179028       0.024998    40.003969
10               DeltaP_2d  0.143114       0.023097    43.294869
9                DeltaP_1d  0.084158       0.017834    56.073137
7       diffPressure9am3p

# Fonction pour un calcul personnalisé de la distance

"Pour les echantillons A et B, sur la dimension i : abs( A(i) - B(i) ) * info mutuelle (i)"

In [37]:
def distance_mi(x,y):
    ma_distance =  np.abs(x-y).dot(info_mutuelle_moy)
    # display("distance_mi_inv(x,y) :"+ str(ma_distance),clear = True)
    return ma_distance

# Simulation avec la distance "distance_mi":

Comme vu plus haut, nous utilisons les paramètres : 

- leaf_size = 30 : Propose un bon compromis 

- weights = 'distance'

- metric = distance_mi

- algorithm = 'auto'.

- k = 4 : On fait un premier essai avec 4 voisins afin de mesurer la durée de la simulation.

Est-ce que 'brute force' sera utilisé ? 'peu de données' ? ==> faux, metric = 'precomputed' ? ==> faux, 'D>15' ? ==> faux (D=15), 'k>N/2' ? ==> faux, le metric utilisée n'est pas dans la liste des metriques valides pour kd_tree ou ball_tree ? ==> VRAI.

Donc 'brute force' sera utilisé.

La 'metric' utilisée est une fonction que nous avons créée. Donc l'algorithme retenu par KNN sera automatiquement 'brute force'.

D'un point de vue technique, la fonction accepte en entrée deux vecteurs (Series) dont les coordonnées sont les variables (colonnes) de X_train ou de X_test. La fonction renvoie un scalaire (float).

In [38]:
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

params = {}
params['nb_voisins'] = [4]
params['metric'] = ['distance_mi']
params['weights'] = ['distance'] 

y_predict = pd.DataFrame()

temps_traitement = []
hdebut = datetime.now()
print("Début traitement : ",hdebut)
temps_traitement.append(["Début traitement",hdebut])

for i in params['nb_voisins'] :
    for dist in params['metric']:
        for w in params['weights'] :
            display('calcul en cours : k='+str(i)+'--metric='+str(dist)+'--weights='+str(w),clear = True)
            #initialisation du modèle modèle :
            knn_mi = KNeighborsClassifier(n_neighbors = i, weights = w, algorithm = "auto", leaf_size = 30, p = 2,
                                              metric = distance_mi, metric_params = None, n_jobs = None)
        
            knn_mi.fit(X_train_minmax,y_train)
            print("fit terminé",datetime.now())
            
            y_p = knn_mi.predict(X_test_minmax)
            print("y_p et y_prob terminés",datetime.now())
                             
            ct = pd.crosstab(y_test,y_p,rownames=['Realité'],colnames=['Prédiction'])
            tp = ct.iloc[1,1]
            tn = ct.iloc[0,0]
            fp = ct.iloc[0,1]
            fn = ct.iloc[1,0]
            
            acc = Accuracy(tp,tn,fp,fn)
            pre = Precision(tp,fp)
            rec = Recall(tp,fn)
            f1s = F1score(rec,pre)
            spe = Specificity(tn,fp)

hfin = datetime.now()

temps_traitement.append(["Fin traitement",hfin])
temps_traitement.append(["Durée traitement",hfin-hdebut])

warnings.filterwarnings("default", category=FutureWarning)

ct

'calcul en cours : k=4--metric=distance_mi--weights=distance'

fit terminé 2023-07-16 03:17:16.449236
y_p et y_prob terminés 2023-07-16 04:38:23.925133


Prédiction    0.0   1.0
Realité                
0.0         17330  1260
1.0          2385  2848

In [39]:
hsav = datetime.now()
temps_traitement.append(["Fin sauvegarde",hsav])
temps_traitement.append(["Durée sauvegarde",hsav - hfin])

print("acc:",acc)
print("pre:",pre)
print("rec:",rec) 
print("f1s:",f1s)
print("spe:",spe)

pd.DataFrame(data=temps_traitement,columns =["Type","Valeur"]).head(10)

acc: 0.8469965999244428
pre: 0.6932814021421616
rec: 0.5442384865278043
f1s: 0.6097848196124611
spe: 0.9322216245293168


Type                      Valeur
0  Début traitement  2023-07-16 03:17:16.433223
1    Fin traitement  2023-07-16 04:38:23.958088
2  Durée traitement              1:21:07.524865
3    Fin sauvegarde  2023-07-16 07:32:00.173940
4  Durée sauvegarde              2:53:36.215852

# KNN Simulations basée sur Mutual Information avec pondération du dataset

In [41]:
print("Somme de l'info. mutuelle :",sum(info_mutuelle_moy))
print("On obtient 95% de cette information mutuelle à partir de :",sum(info_mutuelle_moy)*0.95)

df_info.sort_values(["Info Mutuelle"], ascending=[False])["Info Mutuelle"].cumsum()

Somme de l'info. mutuelle : 0.6681604911298061
On obtient 95% de cette information mutuelle à partir de : 0.6347524665733157


13    0.119809
6     0.188815
0     0.243969
15    0.292354
16    0.339861
17    0.386933
14    0.431162
18    0.474591
12    0.513474
8     0.540965
1     0.567500
11    0.592497
10    0.615595
9     0.633429
7     0.642716
3     0.649525
23    0.652460
4     0.654974
2     0.657407
27    0.659696
21    0.661610
26    0.662743
25    0.663787
22    0.664762
24    0.665726
20    0.666647
5     0.667516
19    0.668160
Name: Info Mutuelle, dtype: float64

En regardant le cumul de l'information mutuelle, on obtient 95% de toute l'information mutuelle si l'on garde les variables:

    No var cumul MI : Variable 
    13     0.119809 : Humidity
    6      0.188815 : diffTempMinMax
    0      0.243969 : Rainfall
    15     0.292354 : DeltaH_2d
    16     0.339861 : DeltaH_3d
    17     0.386933 : WindDirInfluence
    14     0.431162 : DeltaH_1d
    18     0.474591 : consecutiveRainingDays
    12     0.513474 : diffHimidity9am3pm
    8      0.540965 : Pressure
    1      0.567500 : WindGustSpeed
    11     0.592497 : DeltaP_3d
    10     0.615595 : DeltaP_2d
    9      0.633429 : DeltaP_1d
    7      0.642716 : diffPressure9am3pm

# Création de X_train_minmax_im et de X_test_minmax_im

In [42]:
print(X_train_minmax.shape)
print(X_test_minmax.shape)

(95292, 28)
(23823, 28)


In [43]:
X_train_minmax_im = X_train_minmax*info_mutuelle_moy

X_test_minmax_im = X_test_minmax*info_mutuelle_moy

In [44]:
print(X_train_minmax_im.shape)
print(X_test_minmax_im.shape)

(95292, 28)
(23823, 28)


In [46]:
# utilisation des fonctions prédéfinies sur la pondération des données

# fonction "weights_sqr"
def weights_sqr(d):
    inverse = 1.0 / d.astype(float)
    sqr = np.square(inverse)
    return sqr

# fonction "weights_sqr4"
def weights_sqr4(d):
    sqr2 = weights_sqr(d)
    sqr4 = np.square(sqr2)
    return sqr4

# fonction "weights_sqr8"
def weights_sqr8(d):
    sqr4 = weights_sqr4(d)
    sqr8 = np.square(sqr4)
    return sqr8

# Simulation

In [47]:
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

params = {}
params['nb_voisins'] = np.arange(1,21,1)
params['metric'] = ['manhattan','nan_euclidean','minkowski','chebyshev','cosine','euclidean']
params['weights'] = ['uniform','distance','weights_sqr','weights_sqr4','weights_sqr8']

accuracy = []
precision = []
recall = []
f1score = []
specificity = []

CT = []

y_predict = pd.DataFrame()

temps_traitement = []
hdebut = datetime.now()
temps_traitement.append(["Début traitement",hdebut])

for i in params['nb_voisins'] :
    for dist in params['metric']:
        for w in params['weights'] :
            
            #initialisation du modèle modèle :            
            if w == 'weights_sqr':
                display('calcul en cours : k='+str(i)+'--metric='+str(dist)+'--weights='+str(w),clear = True)
                knn = KNeighborsClassifier(n_neighbors = i, weights = weights_sqr, algorithm = "auto", leaf_size = 30, p = 2,
                                       metric = dist, metric_params = None, n_jobs = None)
            elif w == 'weights_sqr4':
                display('calcul en cours : k='+str(i)+'--metric='+str(dist)+'--weights='+str(w),clear = True)
                knn = KNeighborsClassifier(n_neighbors = i, weights = weights_sqr4, algorithm = "auto",
                                               leaf_size = 30, p = 2, metric = dist, metric_params = None, n_jobs = None)
            elif w == 'weights_sqr8':
                display('calcul en cours : k='+str(i)+'--metric='+str(dist)+'--weights='+str(w),clear = True)
                knn = KNeighborsClassifier(n_neighbors = i, weights = weights_sqr8, algorithm = "auto",
                                               leaf_size = 30, p = 2, metric = dist, metric_params = None, n_jobs = None)
            else :
                display('calcul en cours : k='+str(i)+'--metric='+str(dist)+'--weights='+str(w),clear = True)
                knn = KNeighborsClassifier(n_neighbors = i, weights = w, algorithm = "auto",
                                               leaf_size = 30, p = 2, metric = dist, metric_params = None, n_jobs = None)
                 
            knn.fit(X_train_minmax_im,y_train)
        
        
            y_p = knn.predict(X_test_minmax_im)
            y_prob = knn.predict_proba(X_test_minmax_im)
            
            col = 'pred'+str(i)+'zzz'+str(dist)+'zzz'+str(w)
            y_predict[col] = y_p
            
            col_proba = 'proba0zzz'+str(i)+'zzz'+str(dist)+'zzz'+str(w)
            y_predict[col_proba] = y_prob[:,0]
            
            col_proba = 'proba1zzz'+str(i)+'zzz'+str(dist)+'zzz'+str(w)
            y_predict[col_proba] = y_prob[:,1]
                             
            ct = pd.crosstab(y_test,y_p,rownames=['Realité'],colnames=['Prédiction'])
            tp = ct.iloc[1,1]
            tn = ct.iloc[0,0]
            fp = ct.iloc[0,1]
            fn = ct.iloc[1,0]
            
            acc = Accuracy(tp,tn,fp,fn)
            pre = Precision(tp,fp)
            rec = Recall(tp,fn)
            f1s = F1score(rec,pre)
            spe = Specificity(tn,fp)
            
            CT.append([i,dist,w,tp,tn,fp,fn,acc,pre,rec,f1s,spe])
                             
            accuracy.append([i,dist,w,acc])
            precision.append([i,dist,w,pre])
            recall.append([i,dist,w,rec])
            f1score.append([i,dist,w,f1s])
            specificity.append([i,dist,w,spe])

hfin = datetime.now()

temps_traitement.append(["Fin traitement",hfin])
temps_traitement.append(["Durée traitement",hfin-hdebut])

warnings.filterwarnings("default", category=FutureWarning)

# Sauvegarde des résultats

# pond_var_inf_mut_"
chemin_local = ""
nom_fichier = 'accuracy_pond_var_inf_mut_v1.0.csv'
path = chemin_local+nom_fichier
pd.DataFrame(data=accuracy,columns=['k','metric','weights','valeur']).to_csv(nom_fichier,sep=',')

nom_fichier = 'precision_pond_var_inf_mut_v1.0.csv'
path = chemin_local+nom_fichier
pd.DataFrame(data=precision,columns=['k','metric','weights','valeur']).to_csv(nom_fichier,sep=',')

nom_fichier = 'recall_pond_var_inf_mut_v1.0.csv'
path = chemin_local+nom_fichier
pd.DataFrame(data=recall,columns=['k','metric','weights','valeur']).to_csv(nom_fichier,sep=',')

nom_fichier = 'f1score_pond_var_inf_mut_v1.0.csv'
path = chemin_local+nom_fichier
pd.DataFrame(data=f1score,columns=['k','metric','weights','valeur']).to_csv(nom_fichier,sep=',')

nom_fichier = 'specificity_pond_var_inf_mut_v1.0.csv'
path = chemin_local+nom_fichier
pd.DataFrame(data=specificity,columns=['k','metric','weights','valeur']).to_csv(nom_fichier,sep=',')

nom_fichier = 'y_predict_pond_var_inf_mut_v1.0.csv'
pd.DataFrame(data=y_predict).to_csv(nom_fichier,sep=',')

nom_fichier = 'crosstab_pond_var_inf_mut_v1.0.csv'
pd.DataFrame(data=CT,
             columns=['k','metric','weights','tp','tn','fp','fn',
                      'acc','pre','rec','f1s','spe']).to_csv(nom_fichier,sep=',')

hsav = datetime.now()
temps_traitement.append(["Fin sauvegarde",hsav])
                         
pd.DataFrame(data=temps_traitement,columns =["Type","Valeur"]).head(10)

'calcul en cours : k=20--metric=euclidean--weights=weights_sqr8'

C:\Users\plebert1\AppData\Local\Temp\ipykernel_25804\3240040460.py:52: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  y_predict[col] = y_p
C:\Users\plebert1\AppData\Local\Temp\ipykernel_25804\3240040460.py:55: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  y_predict[col_proba] = y_prob[:,0]
C:\Users\plebert1\AppData\Local\Temp\ipykernel_25804\3240040460.py:58: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all colum

Type                      Valeur
0  Début traitement  2023-07-16 10:43:14.267764
1    Fin traitement  2023-07-16 17:16:55.803436
2  Durée traitement              6:33:41.535672
3    Fin sauvegarde  2023-07-16 17:17:25.051838

# Chargement des données

In [48]:
# charger les indicateurs

chemin_local = ""

nom_fichier = 'accuracy_pond_var_inf_mut_v1.0.csv'
path = chemin_local+nom_fichier
accuracy_v1 = pd.read_csv(nom_fichier,sep=',',index_col=0)

nom_fichier = 'precision_pond_var_inf_mut_v1.0.csv'
path = chemin_local+nom_fichier
precision_v1 = pd.read_csv(nom_fichier,sep=',',index_col=0)

nom_fichier = 'recall_pond_var_inf_mut_v1.0.csv'
path = chemin_local+nom_fichier
recall_v1 = pd.read_csv(nom_fichier,sep=',',index_col=0)

nom_fichier = 'f1score_pond_var_inf_mut_v1.0.csv'
path = chemin_local+nom_fichier
f1score_v1 = pd.read_csv(nom_fichier,sep=',',index_col=0)

nom_fichier = 'specificity_pond_var_inf_mut_v1.0.csv'
path = chemin_local+nom_fichier
specificity_v1 = pd.read_csv(nom_fichier,sep=',',index_col=0)

nom_fichier = 'crosstab_pond_var_inf_mut_v1.0.csv'
path = chemin_local+nom_fichier
crosstab_v1 = pd.read_csv(nom_fichier,sep=',',index_col=0)

accuracy_v1["evaluateur"] = "accuracy_v1"
precision_v1["evaluateur"] = "precision_v1"
recall_v1["evaluateur"] = "recall_v1"
f1score_v1["evaluateur"] = "f1score_v1"
specificity_v1["evaluateur"] = "specificity_v1"

mesures = pd.concat([accuracy_v1,precision_v1,recall_v1,f1score_v1,specificity_v1])

print(accuracy_v1.head(15))
print("\n")

print(precision_v1.head(15))
print("\n")

print(recall_v1.head(15))
print("\n")

print(f1score_v1.head(15))
print("\n")

print(specificity_v1.head(15))
print("\n")

print(crosstab_v1.head(15))

    k         metric       weights    valeur   evaluateur
0   1      manhattan       uniform  0.822482  accuracy_v1
1   1      manhattan      distance  0.822482  accuracy_v1
2   1      manhattan   weights_sqr  0.822482  accuracy_v1
3   1      manhattan  weights_sqr4  0.822482  accuracy_v1
4   1      manhattan  weights_sqr8  0.822482  accuracy_v1
5   1  nan_euclidean       uniform  0.811737  accuracy_v1
6   1  nan_euclidean      distance  0.811737  accuracy_v1
7   1  nan_euclidean   weights_sqr  0.811737  accuracy_v1
8   1  nan_euclidean  weights_sqr4  0.811737  accuracy_v1
9   1  nan_euclidean  weights_sqr8  0.811737  accuracy_v1
10  1      minkowski       uniform  0.811737  accuracy_v1
11  1      minkowski      distance  0.811737  accuracy_v1
12  1      minkowski   weights_sqr  0.811737  accuracy_v1
13  1      minkowski  weights_sqr4  0.811737  accuracy_v1
14  1      minkowski  weights_sqr8  0.811737  accuracy_v1


    k         metric       weights    valeur    evaluateur
0   1      

# Représentation graphiques

In [50]:
print("Affichage des courbes de valeur des indicateurs en fonction du nombre de voisins")

import warnings
from bokeh.util.warnings import BokehUserWarning
# warnings.filterwarnings("ignore", category=FutureWarning)
# warnings.simplefilter(action='ignore', category=BokehUserWarning)
warnings.simplefilter(action = 'ignore',category = UserWarning)



# Code Bokeh sans utiliser de boucle : beaucoup plus rapide et zero lag.

from numpy.lib.utils import source
from bokeh.models.widgets import Panel, Tabs
from bokeh.models import ColumnDataSource
from bokeh.models.tools import HoverTool,LassoSelectTool
from bokeh.models.ranges import DataRange1d
from bokeh.colors import RGB
from bokeh.models import LinearColorMapper
from  bokeh.models import PanTool,ResetTool,HoverTool,WheelZoomTool,SaveTool,BoxZoomTool
from bokeh.models import Legend

params = {}

params['nb_voisins'] = np.arange(1,21,1)
params['metric'] = ['manhattan','nan_euclidean','minkowski','chebyshev','cosine','euclidean']
params['weights'] = ['uniform','distance','weights_sqr','weights_sqr4','weights_sqr8']
    
mes_couleurs = ['magenta','maroon','mediumaquamarine','mediumblue','mediumorchid','mediumpurple','mediumseagreen',
                'mediumslateblue','mediumspringgreen','mediumturquoise','mediumvioletred','midnightblue','gold',
                'goldenrod','navy','grey','darksalmon','red','magenta','maroon','mediumaquamarine','mediumblue',
                'mediumslateblue','mediumspringgreen','mediumturquoise','mediumvioletred','midnightblue','gold',
                'goldenrod','navy','grey']
# 9 metrics & 2 weights = 18 curves = 18 colors
# https://docs.bokeh.org/en/2.4.2/docs/reference/colors.html#bokeh-colors-named

# Attention : au moins UNE couleur de plus que le nb courbes !!!
# on utilise la couleur supplémentaire pour colorer le pt quand la souris passe dessus


source1 = ColumnDataSource(accuracy_v1)
source2 = ColumnDataSource(precision_v1)
source3 = ColumnDataSource(recall_v1)
source4 = ColumnDataSource(f1score_v1)
source5 = ColumnDataSource(specificity_v1)

tt1 = [("k", "@k"),("metric", "@metric"),("weights","@weights"),("valeur", "@valeur")]

#tools='hover,xpan,xwheel_zoom,box_zoom,save,reset'
tools=[HoverTool(),BoxZoomTool(), PanTool(),ResetTool()]

##################### Accuracy
p1 = figure(plot_width=800, plot_height=600,x_axis_label='k', y_axis_label='valeur',title="Accuracy",
           toolbar_location = "below",
           tooltips = tt1,tools = tools)

chgt_couleur1 = []
hover1 = []
legend_item1 = []
g1 = [] # liste des graphiques à dessiner sur la figure

for i,dist in enumerate(accuracy_v1['metric'].unique()):
    for j,w in enumerate(accuracy_v1['weights'].unique()):
        ma_source = ColumnDataSource(accuracy_v1[(accuracy_v1['metric']==dist) & (accuracy_v1['weights']==w)])
        g1.append(p1.line(x='k',y='valeur',source = ma_source,color=mes_couleurs[i+j],
                           line_width=2)) #,legend_label=dist+'--'+w))
        g1.append(p1.circle(x='k',y='valeur',source = ma_source,color=mes_couleurs[i+j],
                            size=6)) #,legend_label=dist+'--'+w))

        # chgt de couleur qd la souris passe sur un pt
        chgt_couleur1.append(p1.circle(x='k',y='valeur',source = ma_source,
                          size=15,
                          alpha=0, # alpha=0 ==> invisible tant que la souris ne va pas sur le point
                          hover_fill_color=mes_couleurs[-1], hover_alpha=0.7)) #,legend_label=dist+'--'+w))

        hover1.append(HoverTool(tooltips=tt1,mode='mouse',renderers=[chgt_couleur1[-1]]))
        legend_item1.append((dist+'--'+w,[g1[-1],g1[-2],chgt_couleur1[-1]]))

p1.legend.visible = False
legend1 = Legend(items=legend_item1,location="center",click_policy="hide")

p1.add_layout(legend1, 'right')

tab1 = Panel(child=p1, title="Accuracy")

##################### Precision
p2 = figure(plot_width=800, plot_height=800,x_axis_label='k', y_axis_label='valeur',title="Precision",
            x_range = p1.x_range,
            toolbar_location = "below",
            tooltips = tt1,tools = tools)

chgt_couleur2 = []
hover2 = []
legend_item2 = []
g2 = [] # liste des graphiques à dessiner sur la figure

for i,dist in enumerate(precision_v1['metric'].unique()):
    for j,w in enumerate(precision_v1['weights'].unique()):
        ma_source = ColumnDataSource(precision_v1[(precision_v1['metric']==dist) & (precision_v1['weights']==w)])
        g2.append(p2.line(x='k',y='valeur',source = ma_source,color=mes_couleurs[i+j],line_width=2))
        g2.append(p2.circle(x='k',y='valeur',source = ma_source,color=mes_couleurs[i+j],size=6))

        # chgt de couleur qd la souris passe sur un pt
        chgt_couleur2.append(p2.circle(x='k',y='valeur',source = ma_source,
                          size=15,
                          alpha=0, # alpha=0 ==> invisible tant que la souris ne va pas sur le point
                          hover_fill_color=mes_couleurs[-1], hover_alpha=0.7))

        hover2.append(HoverTool(tooltips=tt1,mode='mouse',renderers=[chgt_couleur2[-1]]))
        legend_item2.append((dist+'--'+w,[g2[-1],g2[-2],chgt_couleur2[-1]]))

p2.legend.visible = False
legend2 = Legend(items=legend_item2,location="center",click_policy="hide")

p2.add_layout(legend2, 'right')

tab2 = Panel(child=p2, title="Precision")

# ##################### Recall
p3 = figure(plot_width=800, plot_height=800,x_axis_label='k', y_axis_label='valeur',title="Recall",
            x_range = p1.x_range,
            toolbar_location = "below",
            tooltips = tt1,tools = tools)

chgt_couleur3 = []
hover3 = []
legend_item3 = []
g3 = [] # liste des graphiques à dessiner sur la figure

for i,dist in enumerate(recall_v1['metric'].unique()):
    for j,w in enumerate(recall_v1['weights'].unique()):
        ma_source = ColumnDataSource(recall_v1[(recall_v1['metric']==dist) & (recall_v1['weights']==w)])
        g3.append(p3.line(x='k',y='valeur',source = ma_source,color=mes_couleurs[i+j],line_width=2))
        g3.append(p3.circle(x='k',y='valeur',source = ma_source,color=mes_couleurs[i+j],size=6))

        # chgt de couleur qd la souris passe sur un pt
        chgt_couleur3.append(p3.circle(x='k',y='valeur',source = ma_source,
                          size=15,
                          alpha=0, # alpha=0 ==> invisible tant que la souris ne va pas sur le point
                          hover_fill_color=mes_couleurs[-1], hover_alpha=0.7))

        hover3.append(HoverTool(tooltips=tt1,mode='mouse',renderers=[chgt_couleur3[-1]]))
        legend_item3.append((dist+'--'+w,[g3[-1],g3[-2],chgt_couleur3[-1]]))

p3.legend.visible = False
legend3 = Legend(items=legend_item3,location="center",click_policy="hide")

p3.add_layout(legend3, 'right')

tab3 = Panel(child=p3, title="Recall")

# ##################### F1-Score
p4 = figure(plot_width=800, plot_height=800,x_axis_label='k', y_axis_label='valeur',title="F1-Score",
            x_range = p1.x_range,
            toolbar_location = "below",
           tooltips = tt1,tools = tools)

chgt_couleur4 = []
hover4 = []
legend_item4 = []
g4 = [] # liste des graphiques à dessiner sur la figure

for i,dist in enumerate(f1score_v1['metric'].unique()):
    for j,w in enumerate(f1score_v1['weights'].unique()):
        ma_source = ColumnDataSource(f1score_v1[(f1score_v1['metric']==dist) & (f1score_v1['weights']==w)])
        g4.append(p4.line(x='k',y='valeur',source = ma_source,color=mes_couleurs[i+j],line_width=2))
        g4.append(p4.circle(x='k',y='valeur',source = ma_source,color=mes_couleurs[i+j],size=6))

        # chgt de couleur qd la souris passe sur un pt
        chgt_couleur4.append(p4.circle(x='k',y='valeur',source = ma_source,
                          size=15,
                          alpha=0, # alpha=0 ==> invisible tant que la souris ne va pas sur le point
                          hover_fill_color=mes_couleurs[-1], hover_alpha=0.7))

        hover4.append(HoverTool(tooltips=tt1,mode='mouse',renderers=[chgt_couleur4[-1]]))
        legend_item4.append((dist+'--'+w,[g4[-1],g4[-2],chgt_couleur4[-1]]))

p4.legend.visible = False
legend4 = Legend(items=legend_item4,location="center",click_policy="hide")

p4.add_layout(legend4, 'right')

tab4 = Panel(child=p4, title="F1-Score")

# ##################### Specificity
p5 = figure(plot_width=800, plot_height=800,x_axis_label='k', y_axis_label='valeur',title="Specificity",
            x_range = p1.x_range,
            toolbar_location = "below",
           tooltips = tt1,tools = tools)

chgt_couleur5 = []
hover5 = []
legend_item5 = []
g5 = [] # liste des graphiques à dessiner sur la figure

for i,dist in enumerate(specificity_v1['metric'].unique()):
    for j,w in enumerate(specificity_v1['weights'].unique()):
        ma_source = ColumnDataSource(specificity_v1[(specificity_v1['metric']==dist) & (specificity_v1['weights']==w)])
        g5.append(p5.line(x='k',y='valeur',source = ma_source,color=mes_couleurs[i+j],line_width=2))
        g5.append(p5.circle(x='k',y='valeur',source = ma_source,color=mes_couleurs[i+j],size=6))

        # chgt de couleur qd la souris passe sur un pt
        chgt_couleur5.append(p5.circle(x='k',y='valeur',source = ma_source,
                          size=15,
                          alpha=0, # alpha=0 ==> invisible tant que la souris ne va pas sur le point
                          hover_fill_color=mes_couleurs[-1], hover_alpha=0.7))

        hover5.append(HoverTool(tooltips=tt1,mode='mouse',renderers=[chgt_couleur5[-1]]))
        legend_item5.append((dist+'--'+w,[g5[-1],g5[-2],chgt_couleur5[-1]]))
        
p5.legend.visible = False
legend5 = Legend(items=legend_item5,location="center",click_policy="hide")

p5.add_layout(legend5, 'right')

tab5 = Panel(child=p5, title="Specificity")

# ##################### Accuracy en fct du F1-Score

p6 = figure(plot_width=800, plot_height=800,x_axis_label='F1-Score', y_axis_label='Accuracy',
#             title="Accuracy en fct du F1-Score",
            toolbar_location = "below",
           x_range = DataRange1d(bounds='auto'),
           y_range = DataRange1d(bounds='auto'))

acc_f1 = accuracy_v1.copy(deep=True)
acc_f1.rename(columns={'valeur':'accuracy'},inplace=True)

acc_f1['f1score'] = f1score_v1['valeur']
acc_f1['precision'] = precision_v1['valeur']
acc_f1['recall'] = recall_v1['valeur']
acc_f1['specificity'] = specificity_v1['valeur']

acc_f1['id_couleur'] = round(30 * (acc_f1['accuracy'] * acc_f1['f1score']),0).astype(int)
# couleurs arbitraires pour différencier :
# les meilleurs compromis F1Score/Accuracy
# les compromis moyens
# les mauvais compromis

source6 = ColumnDataSource(acc_f1)

ma_cmap = LinearColorMapper(palette=mes_couleurs, 
                             low = min(acc_f1['id_couleur']), 
                             high = max(acc_f1['id_couleur']))
            
p6.circle(x='f1score',y='accuracy',
          fill_color= {"field":"id_couleur", "transform":ma_cmap},
          line_color=None,
          source = source6,
          size=6)

# chgt de couleur qd la souris passe sur un pt
chgt_couleur6 = p6.circle(x='f1score',y='accuracy',source = source6,
                          size=15,
                          alpha=0, # alpha=0 ==> invisible tant que la souris ne va pas sur le point
                          hover_fill_color=mes_couleurs[-1], hover_alpha=0.7)

tt2 = [("k", "@k"),("metric", "@metric"),("weights","@weights"),("f1score", "@f1score"),("accuracy","@accuracy"),
      ("recall","@recall"),("precision","@precision"),("specificity","@specificity")]

hover6 = HoverTool(
        tooltips=tt2,
        mode='mouse',
        renderers=[chgt_couleur6])

p6.add_tools(hover6)

tab6 = Panel(child=p6, title="Accuracy en fct du F1-Score")

# ##################### Affichages

# h = column(row(p1,p2),row(p3,p4),row(p5,p6))
# show(h)

tabs = Tabs(tabs=[tab6,tab1,tab2,tab3,tab4,tab5])

show(tabs)

Affichage des courbes de valeur des indicateurs en fonction du nombre de voisins
